In [193]:
import pyodbc
import pandas as pd
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

### Windows Authentication

In [194]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"
conn = pyodbc.connect(conn_string);
crs = conn.cursor()
conn.autocommit = True

### Create Query

In [195]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [196]:
query = 'USE SampleRetail'

In [197]:
execute_query(conn, query)

Query Succeessful!


#### Introducing and Importing Data from SQL Database

Here is the data how looks like

In [198]:
pd.read_sql("SELECT * FROM sale.order_item", con = conn)

,order_id,item_id,product_id,quantity,list_price,discount
0,1,1,20,1,249.99,0.20
1,1,2,8,2,99.99,0.07
2,1,3,10,2,174.99,0.05
3,1,4,16,2,599.00,0.05
4,1,5,4,1,151.99,0.20
...,...,...,...,...,...,...
4717,1614,2,159,2,117.60,0.07
4718,1614,3,213,2,83.31,0.20
4719,1615,1,197,2,161.99,0.20
4720,1615,2,214,1,159.99,0.07


#### Problem: whether the increase in the discount rate positively impacts the number of orders for the products

As shown below the aggregated table derived from the sql, we are trying to answer whether discounts effect the orders? 

In [199]:
pd.read_sql("""SELECT DISTINCT product_id, discount, 
                SUM(quantity) OVER(PARTITION BY product_id, discount) as total_quantity
                FROM sale.order_item"""
                , con = conn)

,product_id,discount,total_quantity
0,2,0.05,26
1,2,0.07,30
2,2,0.10,30
3,2,0.20,32
4,3,0.05,37
...,...,...,...
844,312,0.10,2
845,313,0.10,3
846,313,0.20,1
847,314,0.20,2


Let's get the table derived from the sql database to pandas dataframe

In [200]:
df = pd.read_sql("""SELECT product_id, discount, quantity 
            FROM sale.order_item 
            ORDER BY product_id, discount"""
            , con = conn)
df

,product_id,discount,quantity
0,2,0.05,2
1,2,0.05,2
2,2,0.05,1
3,2,0.05,1
4,2,0.05,1
...,...,...,...
4717,313,0.10,1
4718,313,0.10,2
4719,313,0.20,1
4720,314,0.20,2


In [201]:
df_copy = df.copy()

#### Solution 1: Disaggregated Approach by using statistical test comparing means

We have 307 products

In [202]:
df["product_id"].nunique()

307

All of the orders contain only 1 or 2 counts of purchases

In [203]:
df.groupby(["product_id", "discount"])["quantity"].nunique().value_counts()

2    468
1    381
Name: quantity, dtype: int64

Unique number of discount levels according to products are joining to data

In [204]:
count_discount = pd.read_sql("""SELECT product_id, COUNT(DISTINCT discount) as count_discount
            FROM sale.order_item
            GROUP BY product_id
            ORDER BY product_id"""
            , con = conn)
count_discount

,product_id,count_discount
0,2,4
1,3,4
2,4,4
3,5,4
4,6,4
...,...,...
302,311,2
303,312,1
304,313,2
305,314,1


Most of the products are offered to sale according to 4 different discount levels

In [205]:
df = df.set_index('product_id').join(other = count_discount.set_index('product_id'), on = "product_id", how = "left", rsuffix = "_other")
df["count_discount"].value_counts()

4    4098
3     316
2     231
1      77
Name: count_discount, dtype: int64

Logs have only one level of discount are dropped from the data due to out of assessment

In [206]:
print(df.shape)
df.drop(axis = 0, index = df[df["count_discount"] < 2].index, inplace = True)
df.shape

(4722, 3)


(4645, 3)

Most of the products according to discount levels have number of orders smaller than 5

In [207]:
pd.cut(df.groupby(["product_id", "discount"]).size().values, bins = [0,5,10,20]).value_counts()

(0, 5]      540
(5, 10]     145
(10, 20]     33
dtype: int64

number of orders by products and discount levels are joining to data

In [208]:
df["count_product_discount"] = df.groupby(["product_id", "discount"])["quantity"].transform('size')
df

,discount,quantity,count_discount,count_product_discount
product_id,,,,
2,0.05,2,4,18
2,0.05,2,4,18
2,0.05,1,4,18
2,0.05,1,4,18
2,0.05,1,4,18
...,...,...,...,...
311,0.10,1,2,1
311,0.20,2,2,1
313,0.10,1,2,2


logs have number of orders smaller than 5 are dropped data for statistical tests

In [209]:
df.drop(axis = 0, index = df[df["count_product_discount"] < 6].index, inplace = True)
df.shape

(2416, 4)

In [210]:
df.reset_index(inplace = True)
df

,product_id,discount,quantity,count_discount,count_product_discount
0,2,0.05,2,4,18
1,2,0.05,2,4,18
2,2,0.05,1,4,18
3,2,0.05,1,4,18
4,2,0.05,1,4,18
...,...,...,...,...,...
2411,101,0.20,2,4,7
2412,101,0.20,2,4,7
2413,101,0.20,1,4,7
2414,101,0.20,2,4,7


Here we have final data prior to statistical tests

In [211]:
print(f"number of products: {df['product_id'].nunique()}")
print(f"number of discounts levels in products to be tested: {len(df.groupby(['product_id', 'discount']).size())}")
print(f"number of orders by discount levels:\n {df['count_discount'].value_counts()}")

number of products: 29
number of discounts levels in products to be tested: 114
number of orders by discount levels:
 4    2378
3      38
Name: count_discount, dtype: int64


#### Checking Assumptions prior to statistical tests

Most of the testing groups have observation smaller than 30

In [213]:
pd.cut(df.groupby(["product_id", "discount"]).size().values, bins = [5,10,20,29,100]).value_counts()

(5, 10]      14
(10, 20]     25
(20, 29]     69
(29, 100]     6
dtype: int64

Checking normality by using One Sample Kolmogorov-Smirnov test.
All of the testing groups are distirbuted non-normal

In [140]:
alpha = 0.05
index = pd.MultiIndex.from_tuples([('','')], names=["product_id", "discount_level"])
group = pd.DataFrame()
group.index = index
for product in df["product_id"].unique():
    for discount in df["discount"].unique():
        series = df[(df["product_id"] == product) & (df["discount"] == discount)]["quantity"]
        if len(series)>0:
            test_result = stats.kstest(series, 'norm').pvalue
            result = 'non-normal' if test_result < alpha else 'normal'
            group.loc[(str(product), str(discount)), 'KS-test'] = result
group["KS-test"].value_counts()

non-normal    114
Name: KS-test, dtype: int64

According to the result of KS test. Kruskal-Wallis test is applied to test the difference of means by discount levels of products whether discounts effect are statistically important.

In [136]:
alpha = 0.10
results = pd.DataFrame()
for product in df["product_id"].unique():
    groups = {category: numeric.tolist() for category, numeric in df[df["product_id"] == product].groupby("discount")["quantity"]}
    test_result = stats.kruskal(*groups.values()).pvalue
    result = 'group means are different' if test_result < alpha else 'group means are not different'
    results.loc[product, "KW"] = result
results["KW"].value_counts()

group means are not different    26
group means are different         3
Name: KW, dtype: int64

In [156]:
aggregated_data = pd.read_sql("""SELECT product_id, discount, SUM(quantity) as sum_quantity,
                                1.0*AVG(1.0*quantity) as avg_quantity
                                FROM sale.order_item
                                GROUP BY product_id, discount
                                ORDER BY product_id, discount"""
            , con = conn)
aggregated_data

,product_id,discount,sum_quantity,avg_quantity
0,2,0.05,26,1.444444
1,2,0.07,30,1.500000
2,2,0.10,30,1.666666
3,2,0.20,32,1.523809
4,3,0.05,37,1.370370
...,...,...,...,...
844,312,0.10,2,2.000000
845,313,0.10,3,1.500000
846,313,0.20,1,1.000000
847,314,0.20,2,2.000000


Discounts have statistically significant effect on only these 3 products out of 29 products tested. 

In [151]:
significance_index = results[results["KW"] == "group means are different"].index
aggregated_data[aggregated_data["product_id"].isin(significance_index)]

,product_id,discount,sum_quantity,avg_quantity
24,8,0.05,41,1.708333
25,8,0.07,28,1.333333
26,8,0.10,32,1.391304
27,8,0.20,24,1.411764
36,11,0.05,27,1.588235
37,11,0.07,45,1.730769
38,11,0.10,42,1.555555
39,11,0.20,37,1.370370
184,48,0.05,16,1.777777
185,48,0.07,8,1.333333


#### Result: To measure the effect of discounts on products, only 29 products can be tested out of 307 and 3 of them found to be statistically significant. Although the mean of purchases in different discount levels are statistically significant, there is no any evidence of whether discounts are positively or negatively effect purchases according to above table. These findings are not suprise because of the limited observation within the discount levels and limited variability in quantity of orders.